# Task 1 - evaluate logistic regression model on FakeNewsCorpus test data


In [ ]:
# import libraries
import pandas as pd
import joblib
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from scipy import sparse
import json
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

In [ ]:
#Load the logistic regression model 
logistic_model = joblib.load('logistic_model.joblib')

In [ ]:
#Load data test set 
X_test_scaled = sparse.load_npz("X_test_scaled.npz")
y_test = pd.read_csv("y_test.csv")

In [ ]:
# Evaluate with test
y_pred = logistic_model.predict(X_test_scaled)

f1 = f1_score(y_test, y_pred, average='weighted')
print("Total F1-score:", f1)

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Task 2 - evaluate the logistic regression model on the LIAR data 

In [ ]:
# Load preprocessed LIAR-dataset
liar_data = pd.read_csv("liar_dataset/liar_full_dataset_preprocessed.csv", usecols=['type', 'content'], dtype=str)

In the liar dataset, the 'content' variable has been cleaned and tokenized - and stop words has 
been removed and all tokens has been stemmed. This has been done with the use of our preprocessing
pipeline. We still need to remove rows where either 'type' or 'content' is NaN, though. And we need to 
remove duplicated rows:

In [ ]:
# Remove data points where either 'type' or 'content' is NaN
liar_data = liar_data[liar_data['type'].notna() & liar_data['content'].notna()]

# Remove duplicates
liar_data = liar_data.drop_duplicates()

In [ ]:
# Map types to 'fake' or 'reliable'
type_mapping_liar = {
    'true': 'reliable', 
    'false': 'fake', 
    'half-true': 'fake', 
    'pants-fire': 'fake', 
    'barely-true': 'reliable',
    'mostly-true': 'reliable'
}

liar_data["label"] = liar_data["type"].map(type_mapping_liar)

In [ ]:
# Load the top_10000_words (top 10000 in FakeNewsCorpus train data)
with open("top_10000_words.json", "r", encoding="utf-8") as fil:
    top_10000_words = json.load(fil)

In [ ]:
# Convert 'content' to Bag-of-Words feature matrix
vectorizer = CountVectorizer(vocabulary=top_10000_words)
X_liar = vectorizer.transform(liar_data['content'])

In [ ]:
# Load the scaler (made in Part 2 on the FakeNewsCorpus training data)
scaler = joblib.load('scaler.pkl')

In [ ]:
# Scale data
X_liar_scaled = scaler.transform(X_liar)

In [ ]:
# Create label object
y_liar = liar_data['label']

In [ ]:
# Make prediction
y_pred_liar = logistic_model.predict(X_liar_scaled)

In [ ]:
# Evaluation on liar data 
f1_liar = f1_score(y_liar, y_pred_liar, average='weighted')
print("Total F1-score:", f1_liar)

print("Classification Report:")
print(classification_report(y_liar, y_pred_liar))

print("Confusion Matrix:")
print(confusion_matrix(y_liar, y_pred_liar))